<a href="https://colab.research.google.com/github/hargettc2015/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/Copy_of_assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [5]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [6]:
df[df.BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS']

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,OTHER,01 ONE FAMILY DWELLINGS,1,4090,37,NaN,A1,1193 SACKET AVENUE,NaN,10461.0,1.0,0.0,1.0,"3,404",1328.0,1925.0,1,A1,0,01/01/2019
8,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,18,NaN,A5,1215 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,01/01/2019
9,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,20,NaN,A5,1211 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,01/01/2019
42,3,OTHER,01 ONE FAMILY DWELLINGS,1,6809,54,NaN,A1,2601 AVENUE R,NaN,11229.0,1.0,0.0,1.0,"3,333",1262.0,1925.0,1,A1,0,01/01/2019
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23029,4,OTHER,01 ONE FAMILY DWELLINGS,1,13215,3,NaN,A2,244-15 135 AVENUE,NaN,11422.0,1.0,0.0,1.0,"3,300",1478.0,1925.0,1,A2,635000,04/30/2019
23031,4,OTHER,01 ONE FAMILY DWELLINGS,1,11612,73,NaN,A1,10919 132ND STREET,NaN,11420.0,1.0,0.0,1.0,"2,400",1280.0,1930.0,1,A1,514000,04/30/2019
23032,4,OTHER,01 ONE FAMILY DWELLINGS,1,11808,50,NaN,A0,135-24 122ND STREET,NaN,11420.0,1.0,0.0,1.0,"4,000",1333.0,1945.0,1,A0,635000,04/30/2019
23033,4,OTHER,01 ONE FAMILY DWELLINGS,1,12295,23,NaN,A1,134-34 157TH STREET,NaN,11434.0,1.0,0.0,1.0,"2,500",1020.0,1935.0,1,A1,545000,04/30/2019


In [7]:
df.tail()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
23035,4,OTHER,01 ONE FAMILY DWELLINGS,1,10965,276,NaN,A5,111-17 FRANCIS LEWIS BLVD,NaN,11429.0,1.0,0.0,1.0,"1,800",1224.0,1945.0,1,A5,510000,04/30/2019
23036,4,OTHER,09 COOPS - WALKUP APARTMENTS,2,169,29,NaN,C6,"45-14 43RD STREET, 3C",NaN,11104.0,0.0,0.0,0.0,0,0.0,1929.0,2,C6,355000,04/30/2019
23037,4,OTHER,10 COOPS - ELEVATOR APARTMENTS,2,131,4,NaN,D4,"50-05 43RD AVENUE, 3M",NaN,11377.0,0.0,0.0,0.0,0,0.0,1932.0,2,D4,375000,04/30/2019
23038,4,OTHER,02 TWO FAMILY DWELLINGS,1,8932,18,NaN,S2,91-10 JAMAICA AVE,NaN,11421.0,2.0,1.0,3.0,"2,078",2200.0,1931.0,1,S2,1100000,04/30/2019
23039,4,OTHER,12 CONDOS - WALKUP APARTMENTS,2,1216,1161,NaN,R2,"61-05 39TH AVENUE, F5",F5,11377.0,1.0,0.0,85.0,"15,151",854.0,1927.0,2,R2,569202,04/30/2019


In [8]:
df.drop(['EASE-MENT', 'ADDRESS', 'APARTMENT_NUMBER', 'NEIGHBORHOOD'], axis=1)

,BOROUGH,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,R4,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,21 OFFICE BUILDINGS,4,812,68,O5,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,21 OFFICE BUILDINGS,4,839,69,O5,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,R4,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,R1,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23035,4,01 ONE FAMILY DWELLINGS,1,10965,276,A5,11429.0,1.0,0.0,1.0,"1,800",1224.0,1945.0,1,A5,510000,04/30/2019
23036,4,09 COOPS - WALKUP APARTMENTS,2,169,29,C6,11104.0,0.0,0.0,0.0,0,0.0,1929.0,2,C6,355000,04/30/2019
23037,4,10 COOPS - ELEVATOR APARTMENTS,2,131,4,D4,11377.0,0.0,0.0,0.0,0,0.0,1932.0,2,D4,375000,04/30/2019
23038,4,02 TWO FAMILY DWELLINGS,1,8932,18,S2,11421.0,2.0,1.0,3.0,"2,078",2200.0,1931.0,1,S2,1100000,04/30/2019


In [9]:
df = df[(df.SALE_PRICE >= 100000) & (df.SALE_PRICE <= 2000000)]
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
65,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,691,1130,NaN,R4,"505 WEST 19TH STREET, 7D",7D,10011.0,1.0,0.0,1.0,"18,975",1077.0,2013.0,2,R4,2000000,01/02/2019
66,1,OTHER,10 COOPS - ELEVATOR APARTMENTS,2,1347,18,NaN,D4,"345 EAST 54TH ST, 3B",NaN,10022.0,0.0,0.0,0.0,0,0.0,1960.0,2,D4,330000,01/02/2019
67,1,UPPER EAST SIDE (79-96),10 COOPS - ELEVATOR APARTMENTS,2,1491,62,NaN,D4,"16 EAST 80TH STREET, 2A",NaN,10075.0,0.0,0.0,0.0,0,0.0,1925.0,2,D4,600000,01/02/2019


In [0]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
cutoff = pd.to_datetime('2019-04-01')
train = (df[df['SALE_DATE'] < cutoff])
test  = (df[df['SALE_DATE'] >= cutoff])

In [0]:
import numpy as np
import pandas as pd
import sys
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import datetime
from datetime import date
from datetime import time
from datetime import datetime



In [12]:
train.shape

(10488, 21)

In [13]:
test.shape

(2983, 21)

In [14]:
train.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,10488.0,4634.901506,3704.645866,1.0,1458.75,3891.5,6793.00,16350.0
LOT,10488.0,336.372235,595.312895,1.0,20.00,48.0,271.00,9002.0
EASE-MENT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP_CODE,10488.0,10820.087719,1014.973572,0.0,10310.00,11211.0,11364.00,11697.0
RESIDENTIAL_UNITS,10488.0,1.160278,3.196940,0.0,0.00,1.0,2.00,155.0
COMMERCIAL_UNITS,10488.0,0.137586,1.360410,-1.0,0.00,0.0,0.00,50.0
TOTAL_UNITS,10488.0,1.486175,5.327476,0.0,0.00,1.0,2.00,156.0
GROSS_SQUARE_FEET,10488.0,1596.800057,6630.346460,0.0,0.00,1134.5,1869.00,244619.0
YEAR_BUILT,10483.0,1838.956883,455.869514,0.0,1924.00,1945.0,1965.00,2018.0
TAX_CLASS_AT_TIME_OF_SALE,10488.0,1.572654,0.696860,1.0,1.00,1.0,2.00,4.0


In [15]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
BOROUGH,10488,5,4,3627,NaT,NaT
TAX_CLASS_AT_PRESENT,10488,10,1,4964,NaT,NaT
NEIGHBORHOOD,10488,11,OTHER,8849,NaT,NaT
BUILDING_CLASS_CATEGORY,10488,40,01 ONE FAMILY DWELLINGS,2517,NaT,NaT
SALE_DATE,10488,80,2019-01-31 00:00:00,287,2019-01-01,2019-03-30
BUILDING_CLASS_AT_PRESENT,10488,96,D4,2118,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,10488,96,D4,2118,NaT,NaT
APARTMENT_NUMBER,2146,827,3A,42,NaT,NaT
LAND_SQUARE_FEET,10463,2004,0,4077,NaT,NaT
ADDRESS,10488,10442,N/A BAY STREET,4,NaT,NaT


In [16]:
train['GROSS_SQUARE_FEET'].value_counts()

0.0       3120
1440.0      64
1600.0      50
1200.0      45
1800.0      44
          ... 
6080.0       1
3085.0       1
1906.0       1
4600.0       1
2567.0       1
Name: GROSS_SQUARE_FEET, Length: 2251, dtype: int64

In [17]:
train.groupby('GROSS_SQUARE_FEET')['SALE_PRICE'].mean()

GROSS_SQUARE_FEET
0.0         5.788181e+05
2.0         7.400000e+05
80.0        6.300000e+05
112.0       2.400000e+05
135.0       5.850000e+05
                ...     
194030.0    5.740000e+05
217991.0    3.054750e+05
224400.0    1.989588e+06
231379.0    1.731025e+06
244619.0    4.800000e+05
Name: SALE_PRICE, Length: 2251, dtype: float64

In [0]:
target = 'SALE_PRICE'
high_card = ['BUILDING_CLASS_CATEGORY', 'NEIGHBORHOOD', 'BLOCK', 'LOT', 'EASE-MENT',
             'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
             'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'BUILDING_CLASS_AT_TIME_OF_SALE',
             'SALE_DATE']
features = train.columns.drop([target] + high_card)

In [0]:
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [20]:
X_train

,BOROUGH,TAX_CLASS_AT_PRESENT,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,TAX_CLASS_AT_TIME_OF_SALE
44,3,1,1.0,0.0,1.0,1
61,4,1,1.0,0.0,1.0,1
65,1,2,1.0,0.0,1.0,2
66,1,2,0.0,0.0,0.0,2
67,1,2,0.0,0.0,0.0,2
...,...,...,...,...,...,...
18132,5,1,1.0,0.0,1.0,1
18134,5,1,1.0,0.0,1.0,1
18138,5,1,2.0,0.0,2.0,1
18144,2,1,2.0,0.0,2.0,1


In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)

In [0]:
X_test = encoder.transform(X_test)

In [23]:
X_train

,BOROUGH_3,BOROUGH_4,BOROUGH_1,BOROUGH_2,BOROUGH_5,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_2,TAX_CLASS_AT_PRESENT_4,TAX_CLASS_AT_PRESENT_2A,TAX_CLASS_AT_PRESENT_1B,TAX_CLASS_AT_PRESENT_1A,TAX_CLASS_AT_PRESENT_2C,TAX_CLASS_AT_PRESENT_1C,TAX_CLASS_AT_PRESENT_2B,TAX_CLASS_AT_PRESENT_1D,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,TAX_CLASS_AT_TIME_OF_SALE
44,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,1
61,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,1
65,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,1.0,2
66,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,2
67,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18132,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,1
18134,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,1
18138,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,2.0,0.0,2.0,1
18144,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,2.0,0.0,2.0,1


In [24]:
X_test

,BOROUGH_3,BOROUGH_4,BOROUGH_1,BOROUGH_2,BOROUGH_5,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_2,TAX_CLASS_AT_PRESENT_4,TAX_CLASS_AT_PRESENT_2A,TAX_CLASS_AT_PRESENT_1B,TAX_CLASS_AT_PRESENT_1A,TAX_CLASS_AT_PRESENT_2C,TAX_CLASS_AT_PRESENT_1C,TAX_CLASS_AT_PRESENT_2B,TAX_CLASS_AT_PRESENT_1D,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,TAX_CLASS_AT_TIME_OF_SALE
18167,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,1.0,2
18170,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,2
18173,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,1.0,2
18174,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,2
18175,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23035,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0,0.0,1.0,1
23036,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,2
23037,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,2
23038,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,2.0,1.0,3.0,1


In [25]:
from sklearn.feature_selection import SelectKBest, f_regression
selector = SelectKBest(score_func=f_regression, k=15)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape

((10488, 15), (2983, 15))

In [26]:
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
BOROUGH_3
BOROUGH_4
BOROUGH_1
BOROUGH_2
BOROUGH_5
TAX_CLASS_AT_PRESENT_2
TAX_CLASS_AT_PRESENT_4
TAX_CLASS_AT_PRESENT_2A
TAX_CLASS_AT_PRESENT_1A
TAX_CLASS_AT_PRESENT_2C
TAX_CLASS_AT_PRESENT_2B
TAX_CLASS_AT_PRESENT_1D
RESIDENTIAL_UNITS
TOTAL_UNITS
TAX_CLASS_AT_TIME_OF_SALE


Features not selected:
TAX_CLASS_AT_PRESENT_1
TAX_CLASS_AT_PRESENT_1B
TAX_CLASS_AT_PRESENT_1C
COMMERCIAL_UNITS


In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')

    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test MAE: ${mae:,.0f} \n')

1 features
Test MAE: $321,301 

2 features
Test MAE: $317,743 

3 features
Test MAE: $307,936 

4 features
Test MAE: $308,422 

5 features
Test MAE: $306,029 

6 features
Test MAE: $305,902 

7 features
Test MAE: $286,802 

8 features
Test MAE: $286,503 

9 features
Test MAE: $286,214 

10 features
Test MAE: $285,747 

11 features
Test MAE: $285,637 

12 features
Test MAE: $285,567 

13 features
Test MAE: $285,576 

14 features
Test MAE: $284,799 

15 features
Test MAE: $284,799 

16 features
Test MAE: $284,706 

17 features
Test MAE: $284,683 

18 features
Test MAE: $284,640 

19 features
Test MAE: $284,640 

